In [ ]:
# Install requests library if not already installed (usually not needed in Colab)
# !pip install requests

import requests
import json # Import json for pretty printing
import sys # To exit the loop gracefully

print("--- IBM Cloud AI Agent Interaction ---")

# --- IMPORTANT: Get your IBM Cloud API Key ---
# 1. Log in to your IBM Cloud account.
# 2. Go to Manage > Access (IAM).
# 3. Click on API keys in the left navigation.
# 4. Click Create an IBM Cloud API key.
# 5. Copy the API key immediately after creation.

# NOTE: For security, never hardcode your API key in shared notebooks.
# This method prompts for input, which is better than hardcoding.
# For truly secure handling, use Colab's Secrets feature (as discussed before)
API_KEY = input("Please enter your IBM Cloud API Key: ").strip()

if not API_KEY:
    print("API Key cannot be empty. Please restart and provide a valid key.")
    sys.exit(1) # Exit the script if API key is not provided

print("\nAttempting to obtain IAM token...")
mltoken = None # Initialize mltoken to None
try:
    token_response = requests.post(
        'https://iam.cloud.ibm.com/identity/token',
        data={"apikey": API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'}
    )
    token_response.raise_for_status() # Raise an exception for bad status codes
    mltoken = token_response.json()["access_token"]
    print("IAM token obtained successfully.")
except requests.exceptions.RequestException as e:
    print(f"Error obtaining IAM token: {e}")
    if 'token_response' in locals():
        print(f"Response: {token_response.text}")
    sys.exit(1) # Exit if token cannot be obtained
except KeyError:
    print("Error: 'access_token' not found in IAM response. Check your API Key.")
    if 'token_response' in locals():
        print(f"IAM Response: {token_response.text}")
    sys.exit(1) # Exit if token is malformed

# ... (your existing code) ...

if mltoken:
    header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
    deployment_url = 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/d775e98a-77c0-4876-9b2b-76266ff7b91d/ai_service_stream?version=2021-05-01'

    print("\n--- Start Chatting (type 'exit' or 'quit' to end) ---")
    conversation_history = []

    while True:
        user_message = input("You: ").strip()

        if user_message.lower() in ['exit', 'quit']:
            print("Exiting chat. Goodbye!")
            break

        if not user_message:
            print("Please enter a message.")
            continue

        conversation_history.append({"content": user_message, "role": "user"})
        payload_scoring = {"messages": conversation_history}

        print(f"Sending message to AI agent to: {deployment_url}") # Debug print
        print(f"Payload: {json.dumps(payload_scoring, indent=2)}") # Debug print payload

        try:
            response_scoring = requests.post(
                deployment_url,
                json=payload_scoring,
                headers={'Authorization': 'Bearer ' + mltoken},
                stream=True
            )

            print(f"Response Status Code: {response_scoring.status_code}") # Debug print status
            response_scoring.raise_for_status() # Raise an exception for bad status codes

           # ... (previous code) ...

            ai_response_content = "" # This variable will store the full AI response
            print("Agent: ", end="") # Print "Agent: " once and then stream the response

            for line in response_scoring.iter_lines():
                if line:
                    decoded_line = line.decode('utf-8')
                    # Removed the DEBUG print from previous suggestion for cleaner output
                    if decoded_line.startswith('data:'):
                        try:
                            json_data = json.loads(decoded_line[5:].strip())
                            if "choices" in json_data and len(json_data["choices"]) > 0:
                                if "delta" in json_data["choices"][0] and "content" in json_data["choices"][0]["delta"]:
                                    content_chunk = json_data["choices"][0]["delta"]["content"]
                                    ai_response_content += content_chunk # <--- Accumulating the content here
                                    print(content_chunk, end="", flush=True) # Printing chunks as they arrive
                        except json.JSONDecodeError:
                            print(f"\nWarning: Could not decode JSON from line: {decoded_line}")
                        except Exception as e:
                            print(f"\nWarning: Error processing line: {e} - {decoded_line}")

            print() # Print a newline at the end of the streamed response

            # After the loop, ai_response_content holds the complete message.
            # You are already adding it to conversation_history here:
            if ai_response_content:
                conversation_history.append({"content": ai_response_content, "role": "assistant"})

# ... (rest of the code) ...
        except requests.exceptions.RequestException as e:
            print(f"\nError during scoring request: {e}")
            print(f"Status Code: {response_scoring.status_code if 'response_scoring' in locals() else 'N/A'}")
            if 'response_scoring' in locals():
                print(f"Response Text (on error): {response_scoring.text}") # Get full error message
            print("An error occurred. Please try again or type 'exit'.")
            if len(conversation_history) > 0 and conversation_history[-1]["role"] == "user":
                conversation_history.pop()
        except Exception as e:
            print(f"\nAn unexpected error occurred: {e}")
            print("An error occurred. Please try again or type 'exit'.")
            if len(conversation_history) > 0 and conversation_history[-1]["role"] == "user":
                conversation_history.pop()

print("\n--- Script Finished ---")

--- IBM Cloud AI Agent Interaction ---
Please enter your IBM Cloud API Key: _Xw6WvZ29X8fdiW7XB5-ctMN4SHsvBe_1E-huZuBg3z8

Attempting to obtain IAM token...
IAM token obtained successfully.

--- Start Chatting (type 'exit' or 'quit' to end) ---
You: hello
Sending message to AI agent to: https://us-south.ml.cloud.ibm.com/ml/v4/deployments/d775e98a-77c0-4876-9b2b-76266ff7b91d/ai_service_stream?version=2021-05-01
Payload: {
  "messages": [
    {
      "content": "hello",
      "role": "user"
    }
  ]
}
Response Status Code: 200
Agent: Hello! How can I assist you with your research today? Please provide me with the specific topic or question you'd like me to explain.
You: i want to research on how will be the ai helps in health care in future
Sending message to AI agent to: https://us-south.ml.cloud.ibm.com/ml/v4/deployments/d775e98a-77c0-4876-9b2b-76266ff7b91d/ai_service_stream?version=2021-05-01
Payload: {
  "messages": [
    {
      "content": "hello",
      "role": "user"
    },
    {
